In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import time
import os
from collections import Counter
import torch
import torch.nn as nn
from torch.optim.optimizer import Optimizer

In [ ]:
#pip install git+https://github.com/lukemelas/pytorch-pretrained-gans

  Cloning https://github.com/lukemelas/pytorch-pretrained-gans to c:\temp\pip-req-build-tggofb3t
  Resolved https://github.com/lukemelas/pytorch-pretrained-gans to commit 2982fdab4e683165e45bc2f4a64c2942a7a3a1b7
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/lukemelas/pytorch-pretrained-gans 'C:\Temp\pip-req-build-tggofb3t'


In [ ]:
#from pytorch_pretrained_gans import make_gan

In [ ]:
#G = make_gan(gan_type='selfconditionedgan', model_name='self_conditioned')

Downloading: "http://selfcondgan.csail.mit.edu/weights/selfcondgan_i_model.pt" to C:\Users\He Zhi Yin/.cache\torch\hub\checkpoints\selfcondgan_i_model.pt
100.0%


Loaded pretrained GAN weights (iteration: 200000.0)


In [8]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, enable=True)

[]


In [24]:
import os
print(os.listdir("../data"))

['ARXIV_V5_CHESTXRAY.pdf', 'BBox_List_2017.csv', 'Data_Entry_2017.csv', 'FAQ_CHESTXRAY.pdf', 'images_001', 'images_002', 'images_003', 'images_004', 'images_005', 'images_006', 'images_007', 'images_008', 'images_009', 'images_010', 'images_011', 'images_012', 'LOG_CHESTXRAY.pdf', 'README_CHESTXRAY.pdf', 'test_list.txt', 'train_val_list.txt']


In [25]:
xray_data = pd.read_csv('../data/Data_Entry_2017.csv')
xray_data.head(5)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [26]:
from glob import glob
#import os # already imported earlier

my_glob = glob('../data/images*/images/*.png')
print('Number of Observations: ', len(my_glob))

Number of Observations:  112120


In [27]:
full_img_paths = {os.path.basename(x): x for x in my_glob}
xray_data['full_path'] = xray_data['Image Index'].map(full_img_paths.get)

In [28]:
xray_data.head(5)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,full_path
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN,../data\images_001\images\00000001_000.png
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN,../data\images_001\images\00000001_001.png
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN,../data\images_001\images\00000001_002.png
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN,../data\images_001\images\00000002_000.png
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN,../data\images_001\images\00000003_000.png


In [29]:
dummy_labels = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 
'Cardiomegaly', 'Nodule', 'Mass', 'Hernia']

for label in dummy_labels:
    xray_data[label] = xray_data['Finding Labels'].map(lambda result: 1.0 if label in result else 0)
display(xray_data)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Edema,Emphysema,Fibrosis,Effusion,Pneumonia,Pleural_Thickening,Cardiomegaly,Nodule,Mass,Hernia
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def create_stratified_sample(df, sample_size=0.2, random_state=42):
    onehot_columns = df.columns[-14:]
    
    original_label_counts = df[onehot_columns].sum().to_dict()
    
    total_labels = sum(original_label_counts.values())
    original_label_freq = {k: v/total_labels for k, v in original_label_counts.items()}
    
    print("Original label distribution:")
    for label, count in original_label_counts.items():
        print(f"{label}: {count} ({original_label_freq[label]*100:.2f}%)")
    
    # Determine final sample size
    if isinstance(sample_size, float) and 0 < sample_size < 1:
        final_sample_size = int(len(df) * sample_size)
    elif isinstance(sample_size, int) and sample_size > 1:
        final_sample_size = min(sample_size, len(df))
    else:
        raise ValueError("sample_size must be a float between 0-1 or an integer > 1")
    
    label_rarity = {col: 1.0 / max(1, df[col].sum()) for col in onehot_columns}
    
    df['rarity_score'] = 0
    for col in onehot_columns:
        df.loc[df[col] == 1, 'rarity_score'] += label_rarity[col]

    rare_sample_size = int(final_sample_size * 0.2)
    rare_samples = df.sort_values('rarity_score', ascending=False).head(rare_sample_size)
    
    remaining_df = df[~df.index.isin(rare_samples.index)]
    
    remaining_df['label_combination'] = ''
    for col in onehot_columns:
        remaining_df['label_combination'] += remaining_df[col].astype(str)
    
    combination_counts = Counter(remaining_df['label_combination'])
    
    for idx, row in remaining_df.iterrows():
        if combination_counts[row['label_combination']] < 10:
            remaining_df.at[idx, 'label_combination'] = 'rare_combination'
    
    remaining_sample_size = final_sample_size - rare_sample_size
    
    if remaining_sample_size > 0:
        _, stratified_samples = train_test_split(
            remaining_df, 
            test_size=remaining_sample_size/len(remaining_df),
            stratify=remaining_df['label_combination'],
            random_state=random_state
        )
    else:
        stratified_samples = pd.DataFrame(columns=df.columns)
    
    sampled_df = pd.concat([rare_samples, stratified_samples])
    
    sampled_df = sampled_df.drop(['rarity_score', 'label_combination'], axis=1, errors='ignore')
    
    return sampled_df


In [66]:
sample_xray_data = create_stratified_sample(xray_data)

Original dataset size: 112120
Original label distribution:
Consolidation: 4667.0 (6.70%)
Infiltration: 19894.0 (28.57%)
Pneumothorax: 5302.0 (7.61%)
Edema: 2303.0 (3.31%)
Emphysema: 2516.0 (3.61%)
Fibrosis: 1686.0 (2.42%)
Effusion: 13317.0 (19.13%)
Pneumonia: 1431.0 (2.06%)
Pleural_Thickening: 3385.0 (4.86%)
Cardiomegaly: 2776.0 (3.99%)
Nodule: 6331.0 (9.09%)
Mass: 5782.0 (8.30%)
Hernia: 227.0 (0.33%)
rarity_score: 13.000000000000002 (0.02%)


C:\Temp\ipykernel_38668\236122887.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.00021427 0.00021427 0.00021427 ... 0.00021427 0.00021427 0.00021427]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[col] == 1, 'rarity_score'] += label_rarity[col]
C:\Temp\ipykernel_38668\236122887.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remaining_df['label_combination'] = ''
C:\Temp\ipykernel_38668\236122887.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [67]:
display(sample_xray_data)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,...,Edema,Emphysema,Fibrosis,Effusion,Pneumonia,Pleural_Thickening,Cardiomegaly,Nodule,Mass,Hernia
63019,00015580_001.png,Cardiomegaly|Edema|Effusion|Fibrosis|Hernia,1,15580,78,F,PA,2992,2991,0.143,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
17611,00004747_000.png,Emphysema|Fibrosis|Hernia|Infiltration|Pleural...,0,4747,82,M,PA,2802,2991,0.143,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
77352,00018999_001.png,Emphysema|Fibrosis|Hernia|Mass,1,18999,64,M,PA,2992,2991,0.143,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
36018,00009507_001.png,Consolidation|Hernia|Infiltration|Pneumonia,1,9507,59,M,PA,2826,2905,0.143,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
36019,00009507_002.png,Effusion|Hernia|Pneumonia,2,9507,59,M,PA,2992,2991,0.143,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47380,00012045_012.png,No Finding,12,12045,43,F,PA,2992,2991,0.143,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10082,00002615_000.png,No Finding,0,2615,59,F,PA,2500,2048,0.171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74597,00018313_000.png,No Finding,0,18313,28,M,AP,2500,2048,0.168,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92062,00022977_013.png,Effusion,13,22977,43,M,AP,3056,2544,0.139,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
